This is a starter kernel to train a YOLOv5 model on the extra images based on this [kernel](https://www.kaggle.com/ayuraj/train-covid-19-detection-using-yolov5).

### Why train a YOLOv5 with extra images?

From this [discussion post](https://www.kaggle.com/c/nfl-health-and-safety-helmet-assignment/discussion/264154#1467422): 

> The helmet predictions we provide in `train_baseline_helmets.csv` and `test_baseline_helmets.csv` are the output of a helmet detection model that was trained on the additional `images/` and labels from `image_labels.csv` (neither part of train or test). The predictions are imperfect but we do provide prediction confidence in the conf column.

* The provided labels in the `image_labels.csv` files are - Helmet, Helmet-Blurred, Helmet-Difficult, Helmet-Sideline, Helmet-Partial. The ground truth lables for bounding boxes in `train_labels.csv` consist only of `Helmet` label. 
* YOLOv5 trained on extra data can be used as a secondary object detector to refine the prediction of a primary object detector trained using frames of the videos.

### What more can this kernel offer?

The training methodology of this kernel can be extended to improve the `train_baseline_helmets.csv` score.

# 🖼️ What is YOLOv5?

YOLO an acronym for 'You only look once', is an object detection algorithm that divides images into a grid system. Each cell in the grid is responsible for detecting objects within itself.

[Ultralytics' YOLOv5](https://github.com/ultralytics/yolov5) ("You Only Look Once") model family enables real-time object detection with convolutional neural networks.

# 🦄 What is Weights and Biases?

<img src="https://i.imgur.com/gb6B4ig.png" width="400" alt="Weights & Biases" />
<!--- @wandbcode{nfl_extra_yolo, v=2} -->

[Weights & Biases](https://wandb.ai/site) (W&B) is a set of machine learning tools that helps you build better models faster. Check out Experiment Tracking with Weights and Biases to learn more.
Weights & Biases is directly integrated into YOLOv5, providing experiment metric tracking, model and dataset versioning, rich model prediction visualization, and more. You can learn more about W&B in this [kernel](https://www.kaggle.com/ayuraj/experiment-tracking-with-weights-and-biases).

# ☀️ Imports and Setup

According to the official Train Custom Data guide, YOLOv5 requires a certain directory structure.
```
/parent_folder
    /dataset
         /images
         /labels
    /yolov5
```
    
* We thus will create a /tmp directory.
* Download YOLOv5 repository and pip install the required dependencies.
* Install the latest version of W&B and login with your wandb account. You can create your free W&B account here.


In [ ]:
%cd ../
!mkdir tmp
%cd tmp

In [ ]:
# Download YOLOv5
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
# Install dependencies
%pip install -qr requirements.txt  # install dependencies

%cd ../
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

#### How to login to W&B?
    
There are two ways you can login to W&B in a Kaggle kernel setting:

* Run a cell with `wandb.login()`. It will ask for the API key, which you can copy + paste in. **This is ideal if you Quick Save your kernel**. 

* You can also use Kaggle Secrets to store your API key and use the code snippet below to login. If you are not familiar with Kaggle Secrets check this [forum post](https://www.kaggle.com/product-feedback/114053). **This is ideal if you do Run and Save All**. 

```
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

# I have saved my API token with "wandb_api" as Label. 
# If you use some other Label make sure to change the same below. 
wandb_api = user_secrets.get_secret("wandb_api") 

wandb.login(key=wandb_api)
```

More on W&B login [here](https://docs.wandb.ai/ref/cli/wandb-login).

In [ ]:
# Install W&B 
!pip install -q --upgrade wandb


# Login 
import wandb
print(wandb.__version__)
wandb.login()

> 📍 Note: W&B comes pre-installed with Kaggle kernel but to ensure the lastest version of W&B use pip install. 

In [ ]:
# Necessary/extra dependencies. 
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# 🦆 Hyperparameters

In [ ]:
%cd ../
TRAIN_PATH = 'input/nfl-health-and-safety-helmet-assignment/images/'
IMG_SIZE = 256
BATCH_SIZE = 16
EPOCHS = 10

print(f'Number of extra images: {len(os.listdir(TRAIN_PATH))}') 

# 🔨 Prepare Dataset

This is the most important section when it comes to training an object detector with YOLOv5. The directory structure, bounding box format, etc must be in the correct order. This section builds every piece needed to train a YOLOv5 model.


* Create train-validation split.
* Create required /dataset folder structure and more the images to that folder.
* Create data.yaml file needed to train the model.
* Create bounding box coordinates in the required YOLO format.

In [ ]:
# Load image level csv file
extra_df = pd.read_csv('input/nfl-health-and-safety-helmet-assignment/image_labels.csv')
print('Number of ground truth bounding boxes: ', len(extra_df))

# Number of unique labels
label_to_id = {label: i for i, label in enumerate(extra_df.label.unique())}
print('Unique labels: ', label_to_id)

# Group together bbox coordinates belonging to the same image. 
image_bbox_label = {} # key is the name of the image, value is a dataframe with label and bbox coordinates. 
for image, df in extra_df.groupby('image'): 
    image_bbox_label[image] = df.reset_index(drop=True)

# Visualize
extra_df.head(5)

# 🍘 Train-validation split

I am using a naive train-validation split. You can device multiple ways to do so. Some ideas are:
* Count the number of bounding boxes in each image and label encode it. Use this to create stratified split. 
* Split based on labels. 
* If you want to do a K-fold training this kernel can be easily modified to do the same. Here's a [kernel that showcase K-fold training using YOLOv5](https://www.kaggle.com/ayuraj/train-yolov5-cross-validation-ensemble-w-b). 

In [ ]:
# Create train and validation split.
train_names, valid_names = train_test_split(list(image_bbox_label), test_size=0.2, random_state=42)
print(f'Size of dataset: {len(image_bbox_label)},\
       training images: {len(train_names)},\
       validation images: {len(valid_names)}')

# 🍚 Prepare Required Folder Structure

The required folder structure for the dataset directory is:

```
/parent_folder
    /dataset
         /images
             /train
             /val
         /labels
             /train
             /val
    /yolov5
```

Note that I have named the directory `nfl_extra`.

In [ ]:
os.makedirs('tmp/nfl_extra/images/train', exist_ok=True)
os.makedirs('tmp/nfl_extra/images/valid', exist_ok=True)

os.makedirs('tmp/nfl_extra/labels/train', exist_ok=True)
os.makedirs('tmp/nfl_extra/labels/valid', exist_ok=True)

# Move the images to relevant split folder.
for img_name in tqdm(train_names):
    copyfile(f'{TRAIN_PATH}/{img_name}', f'tmp/nfl_extra/images/train/{img_name}')

for img_name in tqdm(valid_names):
    copyfile(f'{TRAIN_PATH}/{img_name}', f'tmp/nfl_extra/images/valid/{img_name}')

> 📍 Note: We can also do this without copying the files to a different location. To do so use `*.txt` files with image paths. 


# 🍜 Create .YAML file

The `data.yaml`, is the dataset configuration file that defines

* an "optional" download command/URL for auto-downloading,
* a path to a directory of training images (or path to a *.txt file with a list of training images),
* a path to a directory of validation images (or path to a *.txt file with a list of validation images),
* the number of classes,
* a list of class names

> 📍 Important: For extra images dataset, the bounding box in each image can belong to 5 classes. That's why I have used the number of classes, nc to be 5. YOLOv5 automatically handles the images without any bounding box coordinates.

> 📍 Note: The `data.yaml` is created in the `yolov5/data` directory as required.

In [ ]:
# Create .yaml file 
import yaml

data_yaml = dict(
    train = '../nfl_extra/images/train',
    val = '../nfl_extra/images/valid',
    nc = 5,
    names = list(extra_df.label.unique())
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('tmp/yolov5/data/data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat tmp/yolov5/data/data.yaml

# 🍮 Prepare Bounding Box Coordinated for YOLOv5

For every image with bounding box(es) a .txt file with the same name as the image will be created in the format shown below:

* One row per object.
* Each row is `class x_center y_center width height` format.
* Box coordinates must be in normalized `xywh` format (from 0 - 1). We can normalize by the boxes in pixels by dividing `x_center` and `width` by `image width`, and `y_center` and `height` by `image height`.
* Class numbers are zero-indexed (start from 0).

> 📍 Note: We don't have to remove the images without bounding boxes from the training or validation sets.

In [ ]:
def get_yolo_format_bbox(img_w, img_h, box):
    """
    Convert the bounding boxes in YOLO format.
    
    Input:
    img_w - Original/Scaled image width
    img_h - Original/Scaled image height
    box - Bounding box coordinates in the format, "left, width, top, height"
    
    Output:
    Return YOLO formatted bounding box coordinates, "x_center y_center width height".
    """
    w = box.width # width 
    h = box.height # height
    xc = box.left + int(np.round(w/2)) # xmin + width/2
    yc = box.top + int(np.round(h/2)) # ymin + height/2

    return [xc/img_w, yc/img_h, w/img_w, h/img_h] # x_center y_center width height
    
# Iterate over each image and write the labels and bbox coordinates to a .txt file. 
for img_name, df in tqdm(image_bbox_label.items()):
    # open image file to get the height and width 
    img = cv2.imread(TRAIN_PATH+'/'+img_name)
    height, width, _ = img.shape 
    
    # iterate over bounding box df
    bboxes = []
    for i in range(len(df)):
        # get a row
        box = df.loc[i]
        # get bbox in YOLO format
        box = get_yolo_format_bbox(width, height, box)
        bboxes.append(box)
    
    if img_name in train_names:
        img_name = img_name[:-4]
        file_name = f'tmp/nfl_extra/labels/train/{img_name}.txt'
    elif img_name in valid_names:
        img_name = img_name[:-4]
        file_name = f'tmp/nfl_extra/labels/valid/{img_name}.txt'
        
    with open(file_name, 'w') as f:
        for i, bbox in enumerate(bboxes):
            label = label_to_id[df.loc[i].label]
            bbox = [label]+bbox
            bbox = [str(i) for i in bbox]
            bbox = ' '.join(bbox)
            f.write(bbox)
            f.write('\n')

In [ ]:
# Extra utility function that can be used for inference. 
def convert_yolo_bbox(img_w, img_h, box):
    """
    Input:
    img_w - Original/Scaled image width
    img_h - Original/Scaled image height
    box - YOLO formatted bbox coordinates in the format, "x_center, y_center, width, height"
    
    Output:
    Return bounding box coordinates in the format, "left, width, top, height"
    """
    xc, yc = int(np.round(box[0]*img_w)), int(np.round(box[1]*img_h))
    w, h = int(np.round(box[2]*img_w)), int(np.round(box[3]*img_h))

    left = xc - int(np.round(w/2))
    top = yc - int(np.round(h/2))

    return [left, top, w, h]

# 🚅 Train with W&B

In [ ]:
%cd tmp/yolov5/

```
--img {IMG_SIZE} \ # Input image size.
--batch {BATCH_SIZE} \ # Batch size
--epochs {EPOCHS} \ # Number of epochs
--data data.yaml \ # Configuration file
--weights yolov5s.pt \ # Model name
--save_period 1\ # Save model after interval
--project kaggle-siim-covid # W&B project name
```

In [ ]:
!python train.py --img 720 \
                 --batch 16 \
                 --epochs 10 \
                 --data data.yaml \
                 --weights yolov5s.pt \
                 --save_period 1\
                 --project nfl-extra

## Interactively debug your model performance.

The bounding box debugger can help you visually debug your YOLOv5 model performance. 

![Animation550.gif](https://i.postimg.cc/KjFYSPsS/Animation550.gif)

## Check out the W&B Dashboard with useful metrics.

### [W&B Dashboard $\rightarrow$](https://wandb.ai/ayush-thakur/nfl-extra/runs/3fmveagt)

![Animation551.gif](https://i.postimg.cc/Hn49dtYQ/Animation551.gif)

# 📍 Next Steps

* I am using full image resolution but training YOLOv5s. To get the most out of the data train a larger YOLOv5 model.
* Play with different image sizes. 
* Find the best, hyperparameters using a train-validation split and use that set of hyperparams to train on full dataset. 
* You can ensemble the predictions with the baseline predictions. 